In [ ]:
pip install SPARQLWrapper

In [1]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")

In [3]:
def getQidLabel(label):
    sparql.setQuery("""
    SELECT ?item WHERE {
      ?item rdfs:label \""""+label+"""\"@en
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    result =results["results"]["bindings"][0]["item"]["value"]
    quid = result.split('/')
    return quid[len(quid)-1]
    

In [4]:
def makelabeldict(Qid):
    sparql.setQuery("""
    select distinct ?propertyLabel ?subjectLabel {
      values (?item) {(wd:"""+Qid+""")}
      ?subject ?predicate ?item .
      ?property wikibase:directClaim ?predicate
      service wikibase:label { bd:serviceParam wikibase:language "en" }
    }Limit 1000
    """)
    #/wdt:P31/wdt:P279 , result["itemDescription"]["value"]
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        if(len(result["subjectLabel"]["value"])<10 and result["subjectLabel"]["value"][0] != 'Q'):
            mydict.append((result["propertyLabel"]["value"] , result["subjectLabel"]["value"]))
    return mydict

In [5]:
def getdescription(Qid):
    sparql.setQuery("""
    SELECT ?itemDescription WHERE  
    { VALUES ?item {wd:"""+Qid+"""}   SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
    }
    """)
    #/wdt:P31/wdt:P279 , result["itemDescription"]["value"]
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        try:
            mydict.append(result['itemDescription']['value'])
        except:
            pass
    return mydict

In [6]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        length=len(result["itemLabel"]["value"])
        first=result["itemLabel"]["value"][0]
        rest=result["itemLabel"]["value"][1:length]
        if(not (first=='Q' and rest.isnumeric())):
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [7]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q3624078']


In [8]:
description=[]
for i in range(len(listofidqHuman)):
    description.append(research(listofidqHuman[i]))

In [9]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [10]:
model_path = "./models/"

In [11]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [12]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [13]:
import unicodedata

In [14]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format(model_path+'GoogleNews-vectors-negative300.bin', binary=True, limit=1000000) 

In [15]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport', 'book', 'film']

In [77]:
nameofspeaker = ""
askedname = False
class Person:
    
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = Person.optimizeJob(unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8'))
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        self.passedTopics=[]  
        
    def refresh(self):
        self.passedTopics=[]
        
    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteBook | '+self.book
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFilm | '+self.film
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\njob: '+self.job+'\nbook: '+self.book+'\nsport: '+self.sport+'\nmusic: '+self.music+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\n')
     
    def optimizeJob(job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
            elif split[i]=='bellfounding':
                split[i]='bellfounder'
            elif split[i]=='coaching':
                split[i]='coach'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob
    
    def deoptimizeJob(job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='manager':
                split[i]='management'
            elif split[i]=='supporter':
                split[i]='support'
            elif split[i]=='therapist':
                split[i]='therapy'
            elif split[i]=='supervisor':
                split[i]='supervision'
            elif split[i]=='scout':
                split[i]='scouting'
            elif split[i]=='bellfounder':
                split[i]='bellfounding'
            elif split[i]=='coach':
                split[i]=='coaching'
        deoptimizedJob=''
        for word in split:
            deoptimizedJob+=word+' '
        deoptimizedJob=deoptimizedJob[:len(deoptimizedJob)-1]
        return deoptimizedJob
    
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.job+'.'
        return text

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,12)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        elif rand==10:
            text+='I like doing '
        elif rand==11:
            text+='I am doing '
        elif rand==12:
            text+='I often do '
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text
        
    def tellYourFavouriteFilm(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite film is '
        elif rand==1:
            text+='I really liked the film '
        elif rand==2:
            text+='I really enjoyed watching the film '
        elif rand==3:
            text+='I like films like '
        text+=self.film
        if rand<4:
            text+='.'
        elif rand==4:
            text+=' was the film that I liked most.'    
        return text
    
    def startTopicFilm(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFilm()+' '
        rand=random.randint(0,3)
        if rand==0:
            text+='What is your favourite film?'
        elif rand==1:
            text+='Which film did you like most?'
        elif rand==2:
            text+='Which films do watch?'
        elif rand==3:
            text+='Which film did you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteBook(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite book is '
        elif rand==1:
            text+='I really like the book '
        elif rand==2:
            text+='I enjoyed reading the book '
        elif rand==3:
            text+='I am currently reading '
        elif rand==4:
            text+='I like the book '
        elif rand==5:
            text+='I find '
        text+=self.book
        if rand<3:
            text+='.'
        elif rand==3:
            text+=' and i really like it.'
        elif rand==4:
            text+=' a lot.'
        elif rand==5:
            text+=' is a pretty good book.'
        return text
    
    def startTopicBook(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteBook()+' '
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite book?'
        elif rand==1:
            text+='What do you like to read?'
        elif rand==2:
            text+='Which kind of books do you like?'
        elif rand==3:
            text+='Do you read books?'
        elif rand==4:
            text+='Which book do you like most?'
        print(name+text)
        return text

    def respond(self, message):
        global nameofspeaker
        global chater
        understood=False
        text=self.name.split(' ')[0]+':   '
        phrases = []
        questions = []
        words = ""
        for letter in message:
            if(letter == '.' or letter == '!'):
                phrases.append(words)
                words = ""
            elif(letter == '?'):
                questions.append(words)
                words = ""
            else:    
                words+=letter
        if not words=='':
            phrases.append(words)
        
        SWITCH_DICT={
            'sport':self.tellYourFavouriteSport,
            'music':self.tellYourFavouriteMusic,
            'job':self.tellYourJob,
            'game':self.tellYourFavouriteGame,
            'food':self.tellYourFavouriteFood,
            'film':self.tellYourFavouriteFilm,
            'book':self.tellYourFavouriteBook
        }
        
        
        #récupérer les sujets des questions et répondre à une
        topicsquestions = []
        if(len(questions)!=0):
            for question in questions:
                guessedTopic=self.guessTopic(question)
                if not guessedTopic==None:
                    understood=True
                    if(not guessedTopic in self.passedTopics):
                        topicsquestions.append(guessedTopic)
        if(len(topicsquestions)!=0):
            rand=random.randint(0,len(topicsquestions)-1)
            topic=topicsquestions[rand]
            self.passedTopics.append(topic)
            text+=SWITCH_DICT.get(topic)()
        
        #récupérer les sujets des autres phrases et répondre à une
        topicsphrases = []    
        if(len(phrases) != 0):
            for phrase in phrases:
                guessedTopic=self.guessTopic(phrase)
                if not guessedTopic==None:
                    understood=True
                    askAbout = random.randint(0,2)
                    mot = self.findBestWordTopic(phrase,guessedTopic)
                    if guessedTopic== 'sport' : chater.sport= mot
                    elif guessedTopic== 'music' : chater.music = mot
                    elif guessedTopic== 'job' : chater.job = mot
                    elif guessedTopic== 'game' : chater.game = mot
                    elif guessedTopic== 'film' : chater.film = mot
                    elif guessedTopic== 'book' : chater.book = mot
                    
                    #desactivé pour le moment
                    if(askAbout == 0):
                        mot = self.findBestWordTopic(phrase,guessedTopic)
                        choice=random.randint(0,8)
                        if choice==0: text += "What is " + mot + "?"
                        elif choice==1: text += "Can you explain " + mot + " for me?"
                        elif choice==2: text += "I don't understand " + mot + ", can you explain it more?"
                        elif choice==3: text += "Really interesting, can you tell me more about " + mot + "?" 
                        elif choice==4: text += " What does " + mot + " mean?"
                        elif choice==5: text += "Where does " + mot + " come from?"     
                        print(text)
                        return 1;
                    elif not guessedTopic in self.passedTopics:
                        topicsphrases.append(guessedTopic)
        if(len(topicsphrases)!=0):
            rand=random.randint(0,len(topicsphrases)-1)
            topic=topicsphrases[rand]
            if(len(topicsquestions)!=0): 
                text+=' '
            self.passedTopics.append(topic)
            text+=SWITCH_DICT.get(topic)()
            
        #répondre qu'on n'a pas compris la question
        if not understood: 
            rand=random.randint(0,3)
            if rand==0:
                text+='What do you mean by that? '
            elif rand==1:
                text+='Sorry, I don\'t understand. '
            elif rand==2:
                text+='Sorry, I didn\'t get your question. '
            elif rand==3:
                text+='I don\'t know what you are talking about. '
            text+='We can talk about sport, music, work, games, food, films and books.'
            print(text)
            return -1
        #on a compris mais plus rien à dire parce qu'on a déjà parlé sur ce sujet
        if len(topicsphrases)==0 and len(topicsquestions)==0:
            return 0
        print(text)
        return 1
        
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def findBestWordTopic(self, text,mytopic):
        bestWord=None
        bestDistance=None
        words=text.split(' ')
        toonear = 0.3
        for word in words:
            try:
                distance=word2vecModel.distance(word, mytopic)
                if (bestWord==None or distance<bestDistance) and distance > toonear:
                    bestWord=word
                    bestDistance=distance
            except:
                continue
        return bestWord
    
    def getTopicAndValueForSpecificQuestion(self, question):
        if self.music in question:
            return ['music', self.music]
        if self.job in question:
            return ['job', self.job]
        if self.food in question:
            return ['food', self.food]
        if self.game in question:
            return ['game', self.game]
        if self.sport in question:
            return ['sport', self.sport]
        if self.book in question:
            return ['book', self.book]
        if self.film in question:
            return ['film', self.film]
        return None
    
    def answerSpecificQuestion(self, topic, value):
        text=self.name.split(' ')[0]+':   '
        description=None
        if topic=='job':
            description=getdescription(getQidLabel(Person.deoptimizeJob(value)))
        else:
            description=getdescription(getQidLabel(value))
        if len(description)==0:
            text += "I'm sorry " + nameofspeaker + ", I don't know how to explain it."
        else:
            isA=' is a '
            if description[0][0] in 'aeiouAEIOU':
                isA=' is an '
            if(topic == 'music'):
                text += "The music genre " + self.music + isA +description[0]+'.'
            elif(topic == 'job'):
                text += "The job " + self.job + " can be described as " +description[0]+'.'
            elif(topic == 'food'):
                text += "The food " + self.food + isA +description[0]+'.'
            elif(topic == 'game'):
                text += "The game " + self.game + isA +description[0]+'.'
            elif(topic == 'sport'):
                text += "The sport " + self.sport + isA +description[0]+'.'
            elif(topic == 'book'):
                text += "The book " + self.book + isA +description[0]+'.'
            elif(topic == 'film'):
                year=description[0][0:4]
                rest=description[0][4:len(description)]
                text+=self.film+' is a'+rest+' which was published in '+year+'.'
        print(text)
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        elif openTopics[rand]=='film':
            doneTopics.append('film')
            return self.startTopicFilm()
        elif openTopics[rand]=='book':
            doneTopics.append('book')
            return self.startTopicBook()
        else:
            return None

In [52]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [53]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [54]:
def getPersonTopicValueForSpecificQuestion(message, persons):
    res=None
    words = ""
    phrases=[]
    questions=[]
    for letter in message:
        if(letter == '.' or letter == '!'):
            phrases.append(words)
            words = ""
        elif(letter == '?'):
            questions.append(words)
            words = ""
        else:    
            words+=letter
    for person in persons:
        for question in questions:
            topicAndValue=person.getTopicAndValueForSpecificQuestion(question)
            if topicAndValue!=None:
                return [person, topicAndValue[0], topicAndValue[1]]
    return None

In [78]:
import time
chater = generatePersons(1)[0]
def startChatInteractive(persons):
    global nameofspeaker
    global chater
    nameofspeaker=input('Please tell us your name: ')
    chater.name = nameofspeaker
    for person in persons: 
        person.introduce()
        time.sleep(1)
    random.shuffle(persons)
    chater.age = input(persons[1].name.split(' ')[0]+':   ' + ' What is your age ? ')
    chater.country = input(persons[2].name.split(' ')[0]+':   ' + ' and where do you come from ? ')
    persons[0].startTopic(persons[0].passedTopics)
    while True:
        message=input(nameofspeaker + ':   ')
        if message=='bye':
            chater.generateDescription()
            for person in persons:
                person.refresh()
            return
        #vérifier s'il y a une question spécifique dans le message qui concerne le sujet d'une personne
        personTopicValue=getPersonTopicValueForSpecificQuestion(message, persons)
        if personTopicValue!=None:
            person=personTopicValue[0]
            topic=personTopicValue[1]
            value=personTopicValue[2]
            person.answerSpecificQuestion(topic, value)
            continue
        if not message=='':
            random.shuffle(persons)
            sleep=True
            find = False
            #rand=random.randint(1, len(persons))
            for person in persons:#[0:rand]:
                if sleep:
                    time.sleep(1)
                res=person.respond(message)
                #On n'a pas compris
                if res==-1:
                    break
                #On a compris mais plus rien à dire sur ce sujet
                elif res==0:
                    sleep=False
                
                elif res==1:
                    find=True
            if not find:
                time.sleep(1)
                message = persons[0].startTopic(persons[0].passedTopics)

In [56]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [73]:
myPersons=generatePersons(4)

In [ ]:
startChat(myPersons, 5)           

In [ ]:
startChatInteractive(myPersons)

Please tell us your name: hugo
Quintus:   Good day, I am Quintus Fabius Maximus Gurges, I am 45 years old and I'm from Qatar.
Ben:   Good day, I am Ben Parker, I'm 38 years old and I'm from Zimbabwe.
Daniel:   Hello, I am Daniel Gomez, I'm 52 years old and I live in Syria.
James:   Hi, my name is James Rogers, I'm 31 years old and I am from South Korea.
Quintus:    What is your age ? 22
Ben:    and where do you come from ? nice
James:   My favourite dish is Pecel. What do you like to eat most?
hugo:   i love sandwiches.
Daniel:   What is sandwiches?
Quintus:   I really enjoy Mie ayam.
Ben:   Really interesting, can you tell me more about sandwiches?
James:   Really interesting, can you tell me more about sandwiches?
hugo:   sandwiches is a kind of braid.
Daniel:   I love Hedik.
James:   Where does sandwiches come from?
Ben:   What is sandwiches?
hugo:   i like judo.
Ben:   I often play air hockey.
Quintus:    What does judo mean?
James:   Where does judo come from?
Daniel:   Really int

In [ ]:
generateDescriptions(myPersons)

In [ ]:
for person in myPersons:
    person.showProperties()